In [20]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pytesseract
import nltk
import pandas as pd
import re
from PIL import Image, ImageFilter
from PIL import Image, ImageEnhance
import os
from datetime import datetime

In [21]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'   

In [40]:
df = pd.read_csv('../files/isp_data.csv')

In [41]:
# delete all records where ISP = "SLT"
df = df[df.ISP != 'SLT']

In [42]:
def convert_date(date_str):
    try:
        # Attempt to parse the date in "month/day/year" format
        date_obj = datetime.strptime(date_str, "%d/%m/%Y")
        # Convert it to "year-month-day" format
        return date_obj.strftime("%Y-%m-%d")
    except ValueError:
        # If the date is already in "year-month-day" format or is not a valid date, return it as is
        return date_str

In [43]:
# Replace 'your_folder_path' with the path to your folder containing images
folder_path = '../Data/Internet_bills_slt'

# Get a list of all files in the folder
file_list = os.listdir(folder_path)

# Filter the list to keep only image files (you can add more extensions if needed)
image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp']
image_files = [f for f in file_list if any(f.lower().endswith(ext) for ext in image_extensions)]




# Loop through the image files and open each one
for image_file in image_files:
    # Construct the full path to the image file
    image_path = os.path.join(folder_path, image_file)


    if image_path in df['image_path'].tolist():
        print(f"Skipping image {image_file} as it's already in the DataFrame.")
        continue
    
    # Open the image using PIL
    try:
        image = Image.open(image_path)
        # print(image_path)

        if image.mode == "P":
        # Convert the image to RGB mode
            image = image.convert("RGB")

        # image.show()


        image = image.filter(ImageFilter.EDGE_ENHANCE)

        # Enhance the brightness, contrast, and saturation
        enhancer = ImageEnhance.Brightness(image)
        image = enhancer.enhance(1.2)  # Increase brightness by 20%

        enhancer = ImageEnhance.Contrast(image)
        image = enhancer.enhance(1.5)  # Increase contrast by 20%

        enhancer = ImageEnhance.Color(image)
        image = enhancer.enhance(1.5)  # Increase saturation by 20%

        # show the image
        # image.show()

        # image = image.filter(ImageFilter.CONTOUR)


        # image.show()


        width, height = image.size


        date_box_coordinates = (105, 284, 713, 661)
        amount_box_coordinates = (89, 958, 1587, 1500)

    

        date_cropped_image = image.crop(date_box_coordinates)
        # date_cropped_image = date_cropped_image.filter(ImageFilter.EDGE_ENHANCE)


        amount_cropped_image = image.crop(amount_box_coordinates)
        # amount_cropped_image = amount_cropped_image.filter(ImageFilter.EDGE_ENHANCE)


        # date_cropped_image.show()
        # amount_cropped_image.show()
    
        text = pytesseract.image_to_string(image)
        text_date = pytesseract.image_to_string(date_cropped_image)
        text_amount = pytesseract.image_to_string(amount_cropped_image)


        # Initialize the isp variable to None
        isp = "Unknown"

        # Check for "Dialog" or "myDialog"
        dialog_pattern = r'\b(?:my)?dialog\b'
        if re.search(dialog_pattern, text, re.IGNORECASE):
            isp = "Dialog"

        # Check for "SLT" or "Sri Lanka Telecom"
        slt_pattern = r'\bslt\b|\bsri\s+lanka\s+telecom\b'
        if re.search(slt_pattern, text, re.IGNORECASE):
            isp = "SLT"

        # Check for "Hutch"
        hutch_pattern = r'\bhutch\b'
        if re.search(hutch_pattern, text, re.IGNORECASE):
            isp = "Hutch"





        # replace | and [ with space
        # text_logo = text_logo.replace('|', '')
        # text_logo = text_logo.replace('[', '')

        text_date = text_date.replace('|', '')
        text_date = text_date.replace('[', '')

        text_amount = text_amount.replace('|', '')
        text_amount = text_amount.replace('[', '')

        # print(text_date)
        # print(text_amount)

        # print("========================================================================================================")


        # Define a regular expression pattern to match the billing date
        billing_date_pattern = r'Billing Date\s+(\d{2}/\d{2}/\d{4})'

        # Search for the billing date using the pattern
        billing_date_match = re.search(billing_date_pattern, text_date)

        # Extract and print the billing date if found
        if billing_date_match:
            billing_date = billing_date_match.group(1)
            # print(f"Billing Date: {billing_date}")
        else:
            print("Date not found: " , text_date)



        # Define a regular expression pattern to match the total charges for the period
        total_charges_pattern = r'Total Charges for the Period\s+([\d,.]+)'

        # Search for the total charges using the pattern
        total_charges_match = re.search(total_charges_pattern, text_amount)

        # Extract and print the total charges if found
        if total_charges_match:
            total_charges = total_charges_match.group(1)
            # Remove commas and convert to float
            total_charges = float(total_charges.replace(',', ''))
            # print(f"Total Charges for the Period: {total_charges:.2f}")
        else:
            print("Total not found: " , text_amount)

        # print("all:" , text_all)


        billing_date = convert_date(billing_date)




        df = df.append({'image_path': image_path,'ISP':isp, 'date': billing_date, 'amount': total_charges}, ignore_index=True)


        print("========================================================================================================")
    
    except Exception as e:
        print(f"Error opening {image_file}: {str(e)}")




Date not found:  Anuradhapura

ice Number '
 0047258595-0267 50000

one eum il

ng Period  01/07/2023 - 31/07/2023 (HOME
362788_1-1-02-1-LKR-101-1-BILL_1.2_!
FITH

MMARY OF INVOICE

es ee (seams, goane

oer aoee Ganguuern) ‘weer

jalance Payments received Charges for the period
3,182.00 J - U 7,500.00 ) + \ 7317.28 =

ILS OF CHARGES FOR THE PERIOD

Total not found:  tric form of the bill has the same legal recognition, effect, validity or enforceability as the original form of the bill, in terms of the Electronic Transactions Act No.19 of 2006,"

omens Coma) ONMLO

ice No. 0047258595-0267 cash _]eneques] Jereait cara .&%,

ccd TRUCE CICELL someresonte
So\ane bate EEREICEEE) oa ane

rd Expiry Date




C:\Users\User\AppData\Local\Temp\ipykernel_29872\2511295752.py:157: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'image_path': image_path,'ISP':isp, 'date': billing_date, 'amount': total_charges}, ignore_index=True)
C:\Users\User\AppData\Local\Temp\ipykernel_29872\2511295752.py:157: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'image_path': image_path,'ISP':isp, 'date': billing_date, 'amount': total_charges}, ignore_index=True)


C:\Users\User\AppData\Local\Temp\ipykernel_29872\2511295752.py:157: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'image_path': image_path,'ISP':isp, 'date': billing_date, 'amount': total_charges}, ignore_index=True)


C:\Users\User\AppData\Local\Temp\ipykernel_29872\2511295752.py:157: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'image_path': image_path,'ISP':isp, 'date': billing_date, 'amount': total_charges}, ignore_index=True)


C:\Users\User\AppData\Local\Temp\ipykernel_29872\2511295752.py:157: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'image_path': image_path,'ISP':isp, 'date': billing_date, 'amount': total_charges}, ignore_index=True)


C:\Users\User\AppData\Local\Temp\ipykernel_29872\2511295752.py:157: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'image_path': image_path,'ISP':isp, 'date': billing_date, 'amount': total_charges}, ignore_index=True)


C:\Users\User\AppData\Local\Temp\ipykernel_29872\2511295752.py:157: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'image_path': image_path,'ISP':isp, 'date': billing_date, 'amount': total_charges}, ignore_index=True)


C:\Users\User\AppData\Local\Temp\ipykernel_29872\2511295752.py:157: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'image_path': image_path,'ISP':isp, 'date': billing_date, 'amount': total_charges}, ignore_index=True)


Date not found:  296269_1-1-02-1-LKR-101-1-BILL_1.9_12:09:44040922

004 475

Ics qoma/semaée GQerdaeid/Account Number
\God gome/alenaviutguisd Qevdsaid/Invoice Number 0044757
Gad Ens/almas &.G sadl/Billing Date 02/09/;

Ga MOQBOO/aAmes &.G srevid/Bill Period

01/08/2022 - 3

BESS 6620H68/ ain

9500 cam) educa 

eel



C:\Users\User\AppData\Local\Temp\ipykernel_29872\2511295752.py:157: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'image_path': image_path,'ISP':isp, 'date': billing_date, 'amount': total_charges}, ignore_index=True)


In [47]:
# get year and month from date
df['year'] = pd.DatetimeIndex(df['date']).year
df['month'] = pd.DatetimeIndex(df['date']).month

In [48]:
df

,image_path,ISP,date,amount,year,month
0,../Data/Internet_bills_dialog/Images\1.png,Dialog,2022-11-04,2890.44,2022,11
1,../Data/Internet_bills_dialog/Images\10.png,Dialog,2023-01-04,2800.00,2023,1
2,../Data/Internet_bills_dialog/Images\11.png,Dialog,2023-01-26,4200.00,2023,1
3,../Data/Internet_bills_dialog/Images\12.png,Dialog,2023-01-26,4000.00,2023,1
4,../Data/Internet_bills_dialog/Images\2.png,Dialog,2022-03-23,500.00,2022,3
5,../Data/Internet_bills_dialog/Images\3.png,Dialog,2022-05-31,800.00,2022,5
6,../Data/Internet_bills_dialog/Images\4.png,Dialog,2022-07-04,2226.56,2022,7
7,../Data/Internet_bills_dialog/Images\5.png,Dialog,2022-07-15,800.00,2022,7
8,../Data/Internet_bills_dialog/Images\6.png,Dialog,2022-08-07,800.00,2022,8
9,../Data/Internet_bills_dialog/Images\7.png,Dialog,2022-08-28,4387.23,2022,8


In [46]:
# save to csv
df.to_csv('../files/isp_data.csv', index=False)